# LogP Modelling
<br>
by Dmitri Stepanov

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from bokeh.io import output_notebook
from bokeh.resources import INLINE
from bokeh.plotting import figure, output_file, show
output_notebook(resources=INLINE)

import warnings
warnings.filterwarnings('ignore')


def data_no_noise(x, a, b, c):
    return a * np.exp(-b * x) + c

x_1 = np.linspace(0, 4, 50)
y_1 = data_no_noise(x_1, 2.5, 1.3, 0.5)

def plot_linear():
    p = figure(plot_width=600, plot_height=600, x_axis_label='x', y_axis_label='y', tools="pan,wheel_zoom,reset")
    p.circle(x_1, y_1, legend_label="", fill_color="#1f78b4",
             line_color="#1f78b4", size=4, fill_alpha=0.2)
    p.output_backend = "svg"
    return show(p)

y_noise = 0.2 * np.random.normal(size=x_1.size)
y_2 = y_1 + y_noise

def plot_linear_wnoise():
    p = figure(plot_width=600, plot_height=600, x_axis_label='x', y_axis_label='y', tools="pan,wheel_zoom,reset")
    p.circle(x_1, y_2, legend_label="", fill_color="#1f78b4", line_color="#1f78b4", size=4, fill_alpha=0.2)
    p.output_backend = "svg"
    return show(p)

def cerate_poly_model(degree):
    model = make_pipeline(PolynomialFeatures(degree), Ridge())
    model.fit(x_1.reshape(50,1), y_2.reshape(50,1))
    y_pred_poly = model.predict(x_1.reshape(50,1))
    return y_pred_poly

def plot_poly_model(degree):
    y_data = cerate_poly_model(degree)
    p = figure(plot_width=600, plot_height=600, x_axis_label='x', y_axis_label='y', tools="pan,wheel_zoom,reset")
    p.line(x_1, y_data.ravel(), legend_label="", line_color="#ef8a62")
    p.circle(x_1, y_2, legend_label="", fill_color="#1f78b4", line_color="#1f78b4", size=4, fill_alpha=0.2)
    p.output_backend = "svg"
    return show(p)

hsdb_data= pd.read_csv('data/hsdb_data_with_descriptors.csv', index_col=0)

def create_model(train_data, train_y, test_data, test_y, selected_model):
    '''fit a single model, test RMSE and MAE
    NB! evaluation with predictions rounded to 3 decimals !'''
    model = selected_model
    
    fitted_model = model.fit(train_data, train_y.values.ravel()) # changed
    print ('{}\n\t\t\t...done '.format(model))
    
    cross_validation(train_data, train_y, selected_model)
    
    model_predictions = np.around(fitted_model.predict(test_data), decimals= 3)

    model_mse = mean_squared_error(test_y, model_predictions)
    model_rmse = np.sqrt(model_mse)
    model_r2= r2_score(test_y, model_predictions)  
    print ('----> RMSE= \t\t{}'.format(model_rmse))

    model_mae = mean_absolute_error(test_y, model_predictions)
    print ('----> MAE= \t\t{}'.format(model_mae))
    print ('----> R^2= \t\t{}'.format(model_r2))
    return fitted_model

def cross_validation(train_data, train_y, model):
    print ('performing cross validation...')
    scores = cross_val_score(model, train_data, train_y.values.ravel(),
                                         scoring="neg_mean_squared_error", cv=10) # changed
    print ('\t\t\t...done')
    rmse_scores = np.sqrt(-scores)
    print ('CV RMSE scores:{}\t SD={}'.format(rmse_scores, rmse_scores.std()))
    print ('----> CV RMSE mean=\t{}'.format(rmse_scores.mean()))
    
def create_models(train_data, train_y, test_data, test_y, models_task):
    '''fir many models, use models_task dictionary for selection'''
    for key, value in models_task.iteritems():
        print ('training {}...'.format(key))
        key = create_model(train_data, train_y, test_data, test_y, value)
        print ('\n\n')
        #return model
        

# Generating data
<br>
$y= 2.5e^{-1.3x}+0.5$ (unknown)

In [ ]:
plot_linear()

# Adding noise
### does not represent all possible data points!

In [ ]:
plot_linear_wnoise()

# Simple linear regression
### create model

In [ ]:
plot_poly_model(1)

# More complex model
### (Second degree polynomial)

In [ ]:
plot_poly_model(2)

# More complex model
### (10th degree polynomial)

In [ ]:
plot_poly_model(10)

# Even more complex model

### (40th degree polynomial)

In [ ]:
plot_poly_model(40)

# Example: modelling logP values

In [ ]:
hsdb_data[0:20]

# Training / test split

In [ ]:
generated_features= list(hsdb_data.columns)
generated_features.remove('NameOfSubstance')
generated_features.remove('hsdb_log_kow')

train_set, test_set = train_test_split(hsdb_data, test_size=0.2, random_state=42)
train_data= train_set[generated_features]
train_logp= train_set[['hsdb_log_kow']]
test_data= test_set[generated_features]
test_logp= test_set[['hsdb_log_kow']]

# Creating a linear model

In [ ]:
linear= create_model(train_data, train_logp, test_data, test_logp, LinearRegression())

# Creating a more complex linear model

In [ ]:
ridge = create_model(train_data, train_logp, test_data, test_logp, Ridge(max_iter= 100, alpha=1))

# Creating a non-linear model

In [ ]:
gradient_boost= create_model(train_data, train_logp, test_data, test_logp, GradientBoostingRegressor(n_estimators=1000))

# Create your own model

### 1. Data

### 2. Algorithms

### 3. Function to train the model

### 4. Train your model!